In [0]:
# pyspark functions
from pyspark.sql.functions import *
# URL processing
import urllib

In [0]:
# Define the path to the Delta table
delta_table_path = "dbfs:/user/hive/warehouse/authentication_credentials"

# Read the Delta table to a Spark DataFrame
aws_keys_df = spark.read.format("delta").load(delta_table_path)

In [0]:
# Get the AWS access key and secret key from the spark dataframe
ACCESS_KEY = aws_keys_df.select('Access key ID').collect()[0]['Access key ID']
SECRET_KEY = aws_keys_df.select('Secret access key').collect()[0]['Secret access key']
# Encode the secrete key
ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")

In [0]:
# AWS S3 bucket name s3://user-12e98c45f0cb-bucket/topics/12e98c45f0cb.pin/partition=0/
AWS_S3_BUCKET = ""
# Mount name for the bucket
MOUNT_NAME = "/mnt/pinterest_mount"
# Source url 
SOURCE_URL = "s3n://{0}:{1}@{2}".format(ACCESS_KEY, ENCODED_SECRET_KEY, AWS_S3_BUCKET)
# Mount the drive
dbutils.fs.mount(SOURCE_URL, MOUNT_NAME)

In [0]:
display(dbutils.fs.ls(MOUNT_NAME))

In [0]:
%sql
--  Disable format checks during the reading of Delta tables
SET spark.databricks.delta.formatCheck.enabled=false

In [0]:
# File location and type
# Asterisk(*) indicates reading all the content of the specified file that have .json extension
file_location = "/mnt/pinterest_mount/topics/<TOPIC NAME>/partition=0/*.json"
file_type = "json"
# Ask Spark to infer the schema
infer_schema = "true"
# Read in JSONs from mounted S3 bucket
df_user = spark.read.format(file_type) \
.option("inferSchema", infer_schema) \
.load(file_location)
display(df_user)

In [0]:
# Assuming your DataFrame is named df
df_user.write \
  .format("parquet") \
  .mode("overwrite") \
  .option("path", "") \
  .saveAsTable("12e98c45f0cb_user")


In [0]:
cleaned_pin1 = df_pin.replace({'User Info Error': None,
                               'No description available Story format':None,
                               'Image src error.':None,
                               'N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e':None,
                               'No Title Data Available':None
                               })

In [0]:
cleaned_pin2 = cleaned_pin1.withColumn('follower_count', 
                                       when(col('follower_count').endswith('k'), regexp_replace(col('follower_count'), 'k', '').cast('int') * 1000)
                                       .when(col('follower_count').endswith('M'), regexp_replace(col('follower_count'), 'M', '').cast('int') * 1000000)
                                       .otherwise(col('follower_count').cast('int')))

In [0]:
cleaned_pin3 = (cleaned_pin2
                #.withColumn("follower_count", cleaned_pin2["follower_count"].cast('float'))
                .withColumn("downloaded", cleaned_pin2["downloaded"].cast('int'))
                .withColumn("ind", cleaned_pin2["index"].cast('int')))

In [0]:
cleaned_pin4 = cleaned_pin3.withColumn('save_location', regexp_replace('save_location','Local save in ', ""))

In [0]:
cleaned_pin5 = cleaned_pin4.select('ind', 'unique_id', 'title', 'description', 'follower_count', 'poster_name', 'tag_list', 'is_image_or_video', 'image_src', 'save_location', 'category')

In [0]:
cleaned_pin5.write.mode("overwrite").saveAsTable("")

In [0]:
clean_geo = df_geo.withColumn("coordinates", array("latitude", "longitude"))
clean_geo = clean_geo.drop("latitude", "longitude")
clean_geo = clean_geo.withColumn("timestamp", to_timestamp("timestamp"))
clean_geo = clean_geo.select("ind", "country", "coordinates", "timestamp")

display(clean_geo)

In [0]:
clean_geo.write.mode("overwrite").saveAsTable("")

In [0]:
clean_user = df_user.withColumn("user_name", concat_ws(" ", "first_name", "last_name"))
clean_user = clean_user.drop("first_name", "last_name")
clean_user = clean_user.withColumn("date_joined", to_timestamp("date_joined"))
clean_user = clean_user.select("ind", "user_name", "age", "date_joined")

display(clean_user)

In [0]:
clean_user.write.mode("overwrite").saveAsTable("")

In [0]:
# Find the most popular catehory in each country

from pyspark.sql.window import Window
from pyspark.sql.functions import rank

df_pin_geo = cleaned_pin5.join(clean_geo, cleaned_pin5.ind == clean_geo.ind, "inner")

popular_category_by_country = df_pin_geo.groupBy("country", "category") \
                                        .agg(count("category").alias("category_count")) \
                                        .orderBy("country", col("category_count").desc())

windowSpec = Window.partitionBy("country").orderBy(col("category_count").desc())

popular_category_by_country = popular_category_by_country.withColumn("rank", rank().over(windowSpec)) \
                                                         .filter(col("rank") == 1) \
                                                         .drop("rank")

display(popular_category_by_country)

In [0]:
# Find the user with the most followers in each country

df_geo_user = clean_user.join(clean_geo, clean_user.ind == clean_geo.ind, "inner")
df_all = df_geo_user.join(cleaned_pin5, clean_user.ind == cleaned_pin5.ind, "inner")

most_followed_user_by_country = df_all.select("country", "user_name", "follower_count").dropDuplicates()

most_followed_user_by_country = most_followed_user_by_country.orderBy(col("follower_count").desc()).limit(1)
display(most_followed_user_by_country)


In [0]:
# What is the most popular category per age group?

df_geo_user = clean_user.join(clean_geo, clean_user.ind == clean_geo.ind, "inner")
df_all = df_geo_user.join(cleaned_pin5, clean_user.ind == cleaned_pin5.ind, "inner")

df_age_group = df_all.withColumn("age_group", when((col("age") >= 18) & (col("age") <= 24), "18-24")
                              .when((col("age") >= 25) & (col("age") <= 35), "25-35")
                              .when((col("age") >= 36) & (col("age") <= 50), "36-50")
                              .when(col("age") > 50, "+50"))

result_df = df_age_group.groupBy("age_group", "category").agg(count("category").alias("category_count"))

from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

windowSpec = Window.partitionBy("age_group").orderBy(col("category_count").desc())

result_df = result_df.withColumn("rank", rank().over(windowSpec)) \
                     .filter(col("rank") == 1) \
                     .drop("rank")

result_df = result_df.select('age_group', 'category', 'category_count')

display(result_df)

In [0]:
# What is the median follower count per age group?
from pyspark.sql.functions import expr


df_geo_user = clean_user.join(clean_geo, clean_user.ind == clean_geo.ind, "inner")
df_all = df_geo_user.join(cleaned_pin5, clean_user.ind == cleaned_pin5.ind, "inner")

df_age_group = df_all.withColumn("age_group", when((col("age") >= 18) & (col("age") <= 24), "18-24")
                              .when((col("age") >= 25) & (col("age") <= 35), "25-35")
                              .when((col("age") >= 36) & (col("age") <= 50), "36-50")
                              .when(col("age") > 50, "+50"))

df_select = df_age_group.select("age_group", "follower_count", "poster_name").drop_duplicates()

median_ages = df_select.groupBy("age_group").agg(
    expr("percentile_approx(follower_count, 0.5)").alias("Median_Age")
)
median_ages.show()

In [0]:
#Find how many users joined each year

# Filter users who joined between 2015 and 2020
users_joined_2015_2020 = clean_user.filter((year("date_joined") >= 2015) & (year("date_joined") <= 2020))

# Group by year and count the number of users
users_count_by_year = users_joined_2015_2020.groupBy(year("date_joined").alias("post_year")).count().withColumnRenamed("count", "number_users_joined")

pin_post_year = clean_geo.withColumn("post_year",year("timestamp"))
output_df = pin_post_year.join(users_count_by_year, on="post_year", how="left")
# output_df = output_df.select("post_year","number_users_join")
display(output_df)

In [0]:
# Find the median follower count for users who joined between 2015 and 2020

pin_task_10 = cleaned_pin5.select("ind", "follower_count")
user_task_10 = clean_user.select("ind", "date_joined")
user_task_10 = user_task_10.withColumn("post_year",year("date_joined"))
task_10 = user_task_10.join(pin_task_10, on="ind", how="inner").drop_duplicates()

median_follow = task_10.groupBy("post_year").agg(
    expr("percentile_approx(follower_count, 0.5)").alias("median_follower_count")
)
median_follow.show()

In [0]:
# Find the median follower count of users that have joined between 2015 and 2020, based on which age group they are part of.

pin_task11 = cleaned_pin5.select('ind', 'follower_count')
user_task11 = clean_user.select('ind', 'age', year('date_joined').alias('post_year'))

user_task11 = user_task11.withColumn("age_group", when((col("age") >= 18) & (col("age") <= 24), "18-24")
                              .when((col("age") >= 25) & (col("age") <= 35), "25-35")
                              .when((col("age") >= 36) & (col("age") <= 50), "36-50")
                              .when(col("age") > 50, "+50"))


task11 = pin_task11.join(user_task11, on='ind', how='inner').drop_duplicates()

median_follow_age_join = task11.groupBy("age_group", "post_year").agg(
    expr("percentile_approx(follower_count, 0.5)").alias("Median_Followers")
)

display(median_follow_age_join)